# 0. 기본 설정 (필수실행, 수정 금지)

In [ ]:
import os
import time
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import ujson as json

from blog_hospital_cralwer import HospitalCrawler
# from app_gnun_crawl_rvws import AppGnUnCrawler
# from cafe_get_each_info import GetEachCafeInfo

---

# 1. 블로그

## 파라미터 설정(수정가능구역)

In [3]:
keyword_list = ['"강남아이원스안과"']

start_date = 20220401
how_many_months = 2

count_word_list = ['실비']

## URL 스크래핑

In [4]:
hc = HospitalCrawler(start_date)

In [6]:
start = time.time()
for keyword in keyword_list:
    hc.crawl_blog_urls(keyword, how_many_months, 133) # 133 is heuristic number.
end = time.time() - start

print(f'elapsed = {end}')

"강남아이원스안과" start
From 20220301 to 20220401
From 20220201 to 20220301
"강남아이원스안과" end
elapsed = 22.61573624610901


## URL 중복제거

In [7]:
start = time.time()
hc.deduplicate_url()
end = time.time() - start

print(f'elapsed = {end}')

elapsed = 5.4979164600372314


## 수집된 URL 기반으로 본문 스크래핑

In [8]:
keyword_dir_list = [x.replace('"', '') for x in keyword_list]

start = time.time()
for keyword_dir in keyword_dir_list:
    hc.crawl_blog_contents(keyword_dir)
end = time.time() - start

print(f'elapsed = {end}')

강남아이원스안과 start
process 강남아이원스안과 from 0 to 500
강남아이원스안과 end
elapsed = 1.588918685913086


## 읽을 수 있도록 인코딩 변환 & 단어 카운트 후 저장

In [9]:
start = time.time()

for keyword_dir in keyword_dir_list:
    contents_dir = f'data/blog/{keyword_dir}/contents'
    file_paths = [f'{contents_dir}/{x}' for x in os.listdir(contents_dir) if 'ipynb' not in x and 'rdbl' not in x]
    
    for path in file_paths:
        blog_contents = pd.read_csv(path, delimiter='|DELIMITER|')
        
        blog_contents_cols = blog_contents.columns[0].split('|DELIMITER|')
        blog_contents_2 = blog_contents[blog_contents.squeeze().map(lambda x: len(x)<50000)]
        blog_contents_series = blog_contents_2.squeeze().map(lambda x: x.split('|DELIMITER|'))
        
        rslt = pd.DataFrame.from_dict(dict(zip(blog_contents_series.index, blog_contents_series.values))).T
        rslt.columns = blog_contents_cols
        
        if len(count_word_list)!=0:
            for count_word in count_word_list:
                rslt[count_word] = rslt['content'].map(lambda x: x.count(count_word))
        
        rslt.to_csv(f"{path.split('.csv')[0]}" + "_rdbl.csv", encoding='utf-8-sig')
        
end = time.time() - start

print(f'elapsed = {end}')    

elapsed = 0.8792011737823486


- 이후 `crawling/data/blog/검색키워드/contents/blog_contents_검색키워드_~~~_rdbl.csv` 다운로드